In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from dataset import pro_encoder, na_encoder, hhblits_encoder
from model import *
import pandas as pd

In [2]:
epochs = 200
batch_size = 128
root_dir = "./"
data_csv = "final_rna.csv"
hhblits_dir = "new/"
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [3]:
## random_split
# df = pd.read_csv(data_csv)
# df = df.sample(frac=1.0)
# df.to_csv('random_rna.csv')
# row = df.shape[0]
# print(row)
# idx_tr = round(row*0.8);idx_ts = round(row*0.1)
# num_spl = [idx_tr,idx_tr+idx_ts,idx_tr+2*idx_ts]
# df.iloc[0:num_spl[0],:].to_csv('train.csv',index=False)
# df.iloc[num_spl[0]:num_spl[1],:].to_csv('val.csv',index=False)
# df.iloc[num_spl[1]:num_spl[2],:].to_csv('test.csv',index=False)

In [4]:
dataset = []
for i in ['train','val','test']:
    filename = i+'.csv'
    df = pd.read_csv(filename)
    pro_ori = pro_encoder(df,'origin'); pro_ori = torch.tensor(pro_ori, dtype=torch.float)
    pro_mut = pro_encoder(df,'mutant'); pro_mut = torch.tensor(pro_mut, dtype=torch.float)
    na = na_encoder(df); na = torch.tensor(na, dtype=torch.float)
    hhb_ori, hhb_mut = hhblits_encoder(hhblits_dir,df)
    hhb_ori, hhb_mut = torch.tensor(hhb_ori, dtype=torch.float), torch.tensor(hhb_mut, dtype=torch.float)
    y = df['ddG(kcal/mol)'].values.tolist(); y = torch.tensor(y, dtype=torch.float)
    dataset.append(TensorDataset(pro_ori, pro_mut, hhb_ori, hhb_mut, na, y))

train_dataset = dataset[0]; val_dataset = dataset[1]; test_dataset = dataset[2]

In [5]:
print(len(train_dataset),len(val_dataset), len(test_dataset))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

900 112 112


# MLP Network

In [6]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# MLPRegressor = MLPRegressor().to(device)

# optimizer = Adam(MLPRegressor.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# MLPRegressor

In [7]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), 'result/mlp_best_all.pth')
#         scheduler.step()
#     f = open('result/mlp_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(MLPRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

In [8]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = MLPRegressor
# model.load_state_dict(torch.load('./result/mlp_best_all.pth'))
# MLPRegressor.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
#     X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = MLPRegressor(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/test'+'_mlp_'+'result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# CNN Network

In [9]:
from torch.optim import Adam
import numpy as np
import math

loss_fn = nn.MSELoss()
CNNRegressor = CNNRegressor().to(device)

optimizer = Adam(CNNRegressor.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
CNNRegressor

CNNRegressor(
  (hidden1): Conv1d(20, 256, kernel_size=(3,), stride=(1,))
  (hidden2): Conv1d(20, 256, kernel_size=(3,), stride=(1,))
  (hidden3): Conv1d(30, 256, kernel_size=(3,), stride=(1,))
  (hidden4): Conv1d(30, 256, kernel_size=(3,), stride=(1,))
  (hidden5): Conv1d(4, 256, kernel_size=(3,), stride=(1,))
  (hidden11): Conv1d(256, 10, kernel_size=(3,), stride=(1,))
  (hidden22): Conv1d(256, 10, kernel_size=(3,), stride=(1,))
  (hidden33): Conv1d(256, 10, kernel_size=(3,), stride=(1,))
  (hidden44): Conv1d(256, 10, kernel_size=(3,), stride=(1,))
  (hidden55): Conv1d(256, 10, kernel_size=(3,), stride=(1,))
  (act): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (linear1): Linear(in_features=40800, out_features=8192, bias=True)
  (linear2): Linear(in_features=8192, out_features=2048, bias=True)
  (linear3): Linear(in_features=2048, out_features=1024, bias=True)
  (linear4): Linear(in_features=1024, out_features=256, bias=True)
  (linear5): Linear(in_features=256, out_features=1

In [12]:
def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
    min_val_loss = float("inf")
    log = []
    for epoch in range(1, epochs+1):
        total_loss = 0.
        best_model = None
        model.train()
        for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
            optimizer.zero_grad()
            X_ori = X_ori.to(device)
            X_mut = X_mut.to(device)
            X_muthhb = X_muthhb.to(device)
            X_orihhb = X_orihhb.to(device)
            X_na = X_na.to(device)
            y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
            print(y_preds)
            y = y.to(device)
            loss = loss_fn(y_preds.ravel(), y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*len(X_ori)
        total_mse = total_loss/len(train_dataset)

        # validate
        model.eval()
        val_losses = 0.
        with torch.no_grad():
            for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
                X_ori = X_ori.to(device)
                X_mut = X_mut.to(device)
                X_muthhb = X_muthhb.to(device)
                X_orihhb = X_orihhb.to(device)
                X_na = X_na.to(device)
                y = y.to(device)
                preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
                val_loss = loss_fn(preds.ravel(), y)
                val_losses += val_loss.item()*len(X_ori)
        val_mse = val_losses/len(val_dataset)

        log.append([epoch,total_mse,val_mse])
        print('| epoch {:3d} | lr {:02.5f} '
                      'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
                      format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
        if (epoch >= 50) and (val_mse < min_val_loss):
            min_val_loss = val_loss
            best_model = model
            torch.save(best_model.state_dict(), './result/cnn_best_all.pth')
        scheduler.step()
    f = open('./result/cnn_log.txt','w')
    for i in log:
        f.write(str(i)+'\n')
    f.close()
TrainModel(CNNRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

tensor([[1.1513],
        [1.1440],
        [1.1471],
        [1.1665],
        [1.1526],
        [1.1304],
        [1.1335],
        [1.1209],
        [1.1116],
        [1.1394],
        [1.1546],
        [1.0946],
        [1.1445],
        [1.1381],
        [1.1559],
        [1.1473],
        [1.1641],
        [1.1567],
        [1.1471],
        [1.1385],
        [1.1438],
        [1.1165],
        [1.1311],
        [1.0743],
        [1.1358],
        [1.1192],
        [1.0888],
        [1.1004],
        [1.1409],
        [1.0788],
        [1.1611],
        [1.1557],
        [1.1499],
        [1.1638],
        [1.1179],
        [1.0953],
        [1.1609],
        [1.1080],
        [1.1398],
        [1.1617],
        [1.1087],
        [1.0848],
        [1.1336],
        [1.1040],
        [1.1180],
        [1.1508],
        [1.1170],
        [1.1120],
        [1.1311],
        [1.1598],
        [1.1314],
        [1.1531],
        [1.0928],
        [1.1247],
        [0.4410],
        [1

KeyboardInterrupt: 

In [11]:
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

test_mse = 0.
test_result = []
model = CNNRegressor
model.load_state_dict(torch.load('result/cnn_best_all.pth'))
model.eval()
for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
    X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
    X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
    X_na = X_na.to(device)
    y = y.to(device)
    preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
    print(preds)
    loss = loss_fn(preds.ravel(), y)
    
    test_result.append(y.tolist())
    test_result.append(preds.tolist())
    test_result.append(loss.tolist())
    test_mse += loss.item()*len(X_ori)
test_mse = test_mse/len(test_dataset)
test_rmse = math.sqrt(test_mse)

file_name = 'result/cnn_test_result.txt'
f = open(file_name,'w')
for i in test_result:
    f.write(str(i)+'\n')
f.close()
print(test_rmse)

tensor([[1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1.1350],
        [1

# LSTM Network

In [ ]:
from torch.optim import Adam
import numpy as np
import math

loss_fn = nn.MSELoss()
LSTMRegressor = LSTMRegressor().to(device)

optimizer = Adam(LSTMRegressor.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
LSTMRegressor

/home/liangjs/.conda/envs/lstm/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


LSTMRegressor(
  (lstm_ori): LSTM(20, 256, dropout=0.1)
  (lstm_mut): LSTM(20, 256, dropout=0.1)
  (lstm_orihhb): LSTM(30, 256, dropout=0.1)
  (lstm_muthhb): LSTM(30, 256, dropout=0.1)
  (lstm_na): LSTM(4, 256, dropout=0.1)
  (linear1): Linear(in_features=256, out_features=10, bias=True)
  (linear2): Linear(in_features=256, out_features=10, bias=True)
  (linear3): Linear(in_features=256, out_features=10, bias=True)
  (linear4): Linear(in_features=256, out_features=10, bias=True)
  (linear5): Linear(in_features=256, out_features=10, bias=True)
  (act): ReLU()
  (linear6): Linear(in_features=41000, out_features=8192, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear7): Linear(in_features=8192, out_features=2048, bias=True)
  (linear8): Linear(in_features=2048, out_features=1024, bias=True)
  (linear9): Linear(in_features=1024, out_features=256, bias=True)
  (linearout): Linear(in_features=256, out_features=1, bias=True)
)

In [ ]:
def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
    min_val_loss = float("inf")
    log = []
    for epoch in range(1, epochs+1):
        total_loss = 0.
        best_model = None
        model.train()
        for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
            optimizer.zero_grad()
            X_ori = X_ori.to(device)
            X_mut = X_mut.to(device)
            X_muthhb = X_muthhb.to(device)
            X_orihhb = X_orihhb.to(device)
            X_na = X_na.to(device)
            y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
            y = y.to(device)
            loss = loss_fn(y_preds.ravel(), y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*len(X_ori)
        total_mse = total_loss/len(train_dataset)

        # validate
        model.eval()
        val_losses = 0.
        with torch.no_grad():
            for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
                X_ori = X_ori.to(device)
                X_mut = X_mut.to(device)
                X_muthhb = X_muthhb.to(device)
                X_orihhb = X_orihhb.to(device)
                X_na = X_na.to(device)
                y = y.to(device)
                preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
                val_loss = loss_fn(preds.ravel(), y)
                val_losses += val_loss.item()*len(X_ori)
        val_mse = val_losses/len(val_dataset)

        log.append([epoch,total_mse,val_mse])
        print('| epoch {:3d} | lr {:02.5f} '
                      'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
                      format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
        if (epoch >= 50) and (val_mse < min_val_loss):
            min_val_loss = val_loss
            best_model = model
            torch.save(best_model.state_dict(), 'result/lstm_best_all.pth')
        scheduler.step()
    f = open('result/lstm_log.txt','w')
    for i in log:
        f.write(str(i)+'\n')
    f.close()
TrainModel(LSTMRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

| epoch   1 | lr 0.00100 train_loss 21.35763 | val_loss 3.20652 | val_rmse 1.79068
| epoch   2 | lr 0.00100 train_loss 2.37638 | val_loss 2.58102 | val_rmse 1.60656
| epoch   3 | lr 0.00100 train_loss 2.36255 | val_loss 2.10985 | val_rmse 1.45253
| epoch   4 | lr 0.00100 train_loss 1.76393 | val_loss 2.02138 | val_rmse 1.42175
| epoch   5 | lr 0.00100 train_loss 1.64985 | val_loss 1.94418 | val_rmse 1.39434
| epoch   6 | lr 0.00100 train_loss 1.52684 | val_loss 2.00181 | val_rmse 1.41485
| epoch   7 | lr 0.00100 train_loss 1.45396 | val_loss 1.96440 | val_rmse 1.40157
| epoch   8 | lr 0.00100 train_loss 1.35043 | val_loss 2.10766 | val_rmse 1.45178
| epoch   9 | lr 0.00100 train_loss 1.56370 | val_loss 1.94879 | val_rmse 1.39599
| epoch  10 | lr 0.00100 train_loss 1.35339 | val_loss 2.36777 | val_rmse 1.53876
| epoch  11 | lr 0.00100 train_loss 1.46831 | val_loss 1.89338 | val_rmse 1.37600
| epoch  12 | lr 0.00100 train_loss 1.22209 | val_loss 1.98759 | val_rmse 1.40982
| epoch  13 | l

In [ ]:
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

test_mse = 0.
test_result = []
model = LSTMRegressor
model.load_state_dict(torch.load('result/lstm_best_all.pth'))
model.eval()
for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
    X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
    X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
    X_na = X_na.to(device)
    y = y.to(device)
    preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
    loss = loss_fn(preds.ravel(), y)
    
    test_result.append(y.tolist())
    test_result.append(preds.tolist())
    test_result.append(loss.tolist())
    test_mse += loss.item()*len(X_ori)
test_mse = test_mse/len(test_dataset)
test_rmse = math.sqrt(test_mse)

file_name = 'result/lstm_test_result.txt'
f = open(file_name,'w')
for i in test_result:
    f.write(str(i)+'\n')
f.close()
print(test_rmse)

0.9102504828791502


# BiLSTM Network

In [ ]:
from torch.optim import Adam
import numpy as np
import math

loss_fn = nn.MSELoss()
BLSTMRegressor = BLSTMRegressor().to(device)

optimizer = Adam(BLSTMRegressor.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
BLSTMRegressor

BLSTMRegressor(
  (lstm_mut): LSTM(20, 256, dropout=0.1, bidirectional=True)
  (lstm_ori): LSTM(20, 256, dropout=0.1, bidirectional=True)
  (lstm_muthhb): LSTM(30, 256, dropout=0.1, bidirectional=True)
  (lstm_orihhb): LSTM(30, 256, dropout=0.1, bidirectional=True)
  (lstm_na): LSTM(4, 256, dropout=0.1, bidirectional=True)
  (linear1): Linear(in_features=512, out_features=10, bias=True)
  (linear2): Linear(in_features=512, out_features=10, bias=True)
  (linear3): Linear(in_features=512, out_features=10, bias=True)
  (linear4): Linear(in_features=512, out_features=10, bias=True)
  (linear5): Linear(in_features=512, out_features=10, bias=True)
  (act): ReLU()
  (linear6): Linear(in_features=41000, out_features=8192, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear7): Linear(in_features=8192, out_features=2048, bias=True)
  (linear8): Linear(in_features=2048, out_features=1024, bias=True)
  (linear9): Linear(in_features=1024, out_features=256, bias=True)
  (linearout): Line

In [ ]:
def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
    min_val_loss = float("inf")
    log = []
    for epoch in range(1, epochs+1):
        total_loss = 0.
        best_model = None
        model.train()
        for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
            optimizer.zero_grad()
            X_ori = X_ori.to(device)
            X_mut = X_mut.to(device)
            X_muthhb = X_muthhb.to(device)
            X_orihhb = X_orihhb.to(device)
            X_na = X_na.to(device)
            y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
            y = y.to(device)
            loss = loss_fn(y_preds.ravel(), y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*len(X_ori)
        total_mse = total_loss/len(train_dataset)

        # validate
        model.eval()
        val_losses = 0.
        with torch.no_grad():
            for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
                X_ori = X_ori.to(device)
                X_mut = X_mut.to(device)
                X_muthhb = X_muthhb.to(device)
                X_orihhb = X_orihhb.to(device)
                X_na = X_na.to(device)
                y = y.to(device)
                preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
                val_loss = loss_fn(preds.ravel(), y)
                val_losses += val_loss.item()*len(X_ori)
        val_mse = val_losses/len(val_dataset)

        log.append([epoch,total_mse,val_mse])
        print('| epoch {:3d} | lr {:02.5f} '
                      'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
                      format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
        if (epoch >= 50) and (val_mse < min_val_loss):
            min_val_loss = val_loss
            best_model = model
            torch.save(best_model.state_dict(), 'result/blstm_best_all.pth')
        scheduler.step()
    f = open('result/blstm_log.txt','w')
    for i in log:
        f.write(str(i)+'\n')
    f.close()
TrainModel(BLSTMRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

| epoch   1 | lr 0.00100 train_loss 49.45978 | val_loss 3.43837 | val_rmse 1.85428
| epoch   2 | lr 0.00100 train_loss 2.42842 | val_loss 2.19490 | val_rmse 1.48152
| epoch   3 | lr 0.00100 train_loss 1.96696 | val_loss 2.61455 | val_rmse 1.61696
| epoch   4 | lr 0.00100 train_loss 2.11748 | val_loss 2.11026 | val_rmse 1.45267
| epoch   5 | lr 0.00100 train_loss 1.70838 | val_loss 1.91446 | val_rmse 1.38364
| epoch   6 | lr 0.00100 train_loss 1.53655 | val_loss 1.91029 | val_rmse 1.38213
| epoch   7 | lr 0.00100 train_loss 1.29872 | val_loss 1.92765 | val_rmse 1.38840
| epoch   8 | lr 0.00100 train_loss 1.27451 | val_loss 1.97948 | val_rmse 1.40694
| epoch   9 | lr 0.00100 train_loss 1.35941 | val_loss 1.91423 | val_rmse 1.38356
| epoch  10 | lr 0.00100 train_loss 1.13704 | val_loss 1.95198 | val_rmse 1.39713
| epoch  11 | lr 0.00100 train_loss 1.03687 | val_loss 2.13880 | val_rmse 1.46246
| epoch  12 | lr 0.00100 train_loss 0.98158 | val_loss 2.24553 | val_rmse 1.49851
| epoch  13 | l

In [ ]:
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

test_mse = 0.
test_result = []
model = BLSTMRegressor.to(device)
model.load_state_dict(torch.load('result/blstm_best_all.pth'))
model.eval()
for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
    X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
    X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
    X_na = X_na.to(device)
    y = y.to(device)
    preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
    loss = loss_fn(preds.ravel(), y)
    
    test_result.append(y.tolist())
    test_result.append(preds.tolist())
    test_result.append(loss.tolist())
    test_mse += loss.item()*len(X_ori)
test_mse = test_mse/len(test_dataset)
test_rmse = math.sqrt(test_mse)

file_name = 'result/blstm_test_result.txt'
f = open(file_name,'w')
for i in test_result:
    f.write(str(i)+'\n')
f.close()
print(test_rmse)

0.8871100818357419


# Trans1

In [ ]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# Trans = Trans1().to(device)

# optimizer = Adam(Trans.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# Trans

In [ ]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             # X_muthhb = X_muthhb.to(device)
#             # X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_na, device)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 # X_muthhb = X_muthhb.to(device)
#                 # X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_na, device)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), 'result/trans1_best_all.pth')
#         scheduler.step()
#     f = open('result/trans1_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(Trans, loss_fn, optimizer, train_loader, val_loader, epochs)

In [ ]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = Trans
# model.load_state_dict(torch.load('result/trans1_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device)
#     X_ori = X_ori.to(device)
#     # X_orihhb = X_orihhb.to(device)
#     # X_muthhb = X_muthhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_ori, X_mut, X_na, device)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/trans1_test_result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# Trans2

In [ ]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# Trans = Trans2().to(device)

# optimizer = Adam(Trans.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# Trans

In [ ]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             # X_ori = X_ori.to(device)
#             # X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_orihhb, X_muthhb, X_na, device)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 # X_ori = X_ori.to(device)
#                 # X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_orihhb, X_muthhb, X_na, device)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), 'result/trans2_best_all.pth')
#         scheduler.step()
#     f = open('result/trans2_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(Trans, loss_fn, optimizer, train_loader, val_loader, epochs)

In [ ]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = Trans
# model.load_state_dict(torch.load('result/trans2_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     # X_mut = X_mut.to(device)
#     # X_ori = X_ori.to(device)
#     X_orihhb = X_orihhb.to(device)
#     X_muthhb = X_muthhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_orihhb, X_muthhb, X_na, device)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/trans2_test_result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)